# Утилиты

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f, t, f_oneway
from typing import List, Tuple
import plotly.graph_objects as go

In [2]:
def calculate_mean(x: np.ndarray) -> float:
    return x.sum() / len(x)


def calculate_var(x: np.ndarray) -> float:
    return 1 / (len(x) - 1) * sum((x - calculate_mean(x)) ** 2)


def calculate_q(*samples: np.ndarray) -> Tuple[float, float]:
    r = len(samples)
    n = 0

    global_mean = 0
    for sample in samples:
        n += len(sample)
        global_mean += sum(sample)
    global_mean /= n

    q1 = 0
    q2 = 0
    for sample in samples:
        mean = calculate_mean(sample)

        q1 += len(sample) * (mean - global_mean) ** 2
        q2 += sum((sample - mean) ** 2)

    return q1, q2

In [3]:
def anova(*samples: np.ndarray, alternative: str = 'two-sided') -> Tuple[float, float]:
    q1, q2 = calculate_q(*samples)

    n = 0
    for sample in samples:
        n += len(sample)

    r = len(samples)

    statistics = q1 / q2 * (n - r) / (r - 1)
    p_value = f.sf(statistics, r - 1, n - r)

    right_tail_p_value = f.sf(statistics, r - 1, n - r)
    left_tail_p_value = f.cdf(statistics, r - 1, n - r)  
    two_sided_p_value = 2 * min(left_tail_p_value, right_tail_p_value)

    if alternative == 'less':
        p_value = left_tail_p_value
    elif alternative == 'greater':
        p_value = right_tail_p_value
    else:
        p_value = two_sided_p_value

    return statistics, p_value

In [4]:
def pairwise_anova(*samples: np.ndarray, alternative: str = 'two-sided'):
    _, q2 = calculate_q(*samples)

    n = 0
    for sample in samples:
        n += len(sample)

    r = len(samples)

    for i, sample_i in enumerate(samples):
        mean_i = calculate_mean(sample_i)
        n_i = len(sample_i)
        for j, sample_j in enumerate(samples[i + 1:], start=i + 1):
            mean_j = calculate_mean(sample_j)
            n_j = len(sample_j)
            statistics = (mean_i - mean_j) / np.sqrt(q2) / np.sqrt(1 / n_i + 1 / n_j) * np.sqrt(n - r)

            right_tail_p_value = t.sf(statistics, n - r)
            left_tail_p_value = t.cdf(statistics, n - r)  
            two_sided_p_value = 2 * min(left_tail_p_value, right_tail_p_value)

            if alternative == 'less':
                p_value = left_tail_p_value
            elif alternative == 'greater':
                p_value = right_tail_p_value
            else:
                p_value = two_sided_p_value

            print(f'{i} - {j}: {statistics}, {p_value}')

# Тестирование

In [5]:
r1 = np.array([1, 3, 2, 1, 0, 2, 1])
r2 = np.array([2, 3, 2, 1, 4])
r3 = np.array([4, 5, 3])

print(f_oneway(r1, r2, r3))
print(anova(r1, r2, r3))
print()
pairwise_anova(r1, r2, r3)

F_onewayResult(statistic=6.513274336283186, pvalue=0.012152949868938764)
(6.513274336283185, 0.02430589973787753)

0 - 1: -1.599225476252115, 0.13575281638761094
0 - 2: -3.5920265682837913, 0.0036996598905966608
1 - 2: -2.1119131116479304, 0.056339625464633176


In [6]:
data = pd.read_excel('addicts0.xls', sheet_name=1)
data.dropna(inplace=True)
grouped_by_educat = data.groupby(by=['educat'])
data

,prcod,intpla,sex,age,educat,curwor,asi1_med,asi2_emp,asi3_alc,asi4_dr,...,ha,se,cravin,rabdru,rubsex,gaf,bdi,sstati,end,endpo
0,4,1,0,18,1,1,0.19,0.70,0.120,0.30,...,1.0,0.0,4.6,1.0,4,55.0,25.0,60,0.0,5.0
1,2,2,1,30,4,1,0.44,0.23,0.006,0.27,...,0.0,0.0,9.7,4.0,5,55.0,39.0,50,0.0,5.0
2,2,1,0,23,2,0,0.50,1.00,0.300,0.30,...,0.0,0.0,9.5,6.0,1,45.0,29.0,55,0.0,2.0
3,4,1,0,20,2,1,0.00,0.80,0.050,0.26,...,1.0,0.0,2.7,11.0,4,40.0,28.0,58,0.0,5.0
4,3,2,0,20,2,0,0.00,0.75,0.780,0.23,...,0.0,0.0,3.0,19.0,4,40.0,28.0,58,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,1,2,0,23,2,0,0.10,0.59,0.000,0.25,...,1.0,0.0,4.5,5.0,2,41.0,17.0,47,0.0,2.0
276,4,2,0,25,2,0,0.00,1.00,0.000,0.25,...,1.0,0.0,0.8,6.0,6,50.0,21.0,45,0.0,2.0
277,1,2,0,24,2,0,0.00,1.00,0.210,0.21,...,0.0,1.0,1.5,8.0,5,50.0,12.0,40,1.0,1.0
278,1,2,1,18,2,0,0.00,1.00,0.020,0.29,...,0.0,1.0,4.0,14.0,5,45.0,22.0,62,0.0,5.0


In [7]:
samples = []
for _, group in grouped_by_educat['asi2_emp']:
    samples.append(group.to_numpy())

print(f_oneway(*samples))
print(anova(*samples))
print()
pairwise_anova(*samples)

F_onewayResult(statistic=3.6219370194494593, pvalue=0.013644080153473894)
(3.621937019449463, 0.027288160306946883)

0 - 1: -0.7429843415631653, 0.45814193561416194
0 - 2: 1.3999330767866673, 0.1626898927610867
0 - 3: 1.180849579443486, 0.2387093963282501
1 - 2: 2.7967669021356905, 0.005535571476671078
1 - 3: 1.8820689756407283, 0.060910157665962736
2 - 3: 0.1544383849869198, 0.8773803286789008


# asi1_med

In [8]:
samples = []
fig = go.Figure()
for name, group in grouped_by_educat['asi1_med']:
    group = group.to_numpy()
    samples.append(group)
    fig.add_box(y=group, name=name)
    print(f'{name}: {len(group)}, {calculate_mean(group)}, {calculate_var(group)}')

fig.update_layout(showlegend=False, margin={'l': 5, 'r': 5, 't': 5, 'b': 5})
fig.write_image('box_plot.svg')
fig.show()

1: 21, 0.40380952380952373, 0.06238476190476192
2: 216, 0.22976851851851857, 0.07027110895779486
3: 26, 0.28807692307692306, 0.08776815384615384
4: 9, 0.21333333333333332, 0.06587499999999999


Unsupported

In [10]:
print(f_oneway(*samples))
print(anova(*samples))

F_onewayResult(statistic=2.9696259894659254, pvalue=0.03235933623199457)
(2.969625989465925, 0.06471867246399045)


In [11]:
pairwise_anova(*samples)

0 - 1: 2.853804089280608, 0.00465715034762881
0 - 2: 1.478472761829135, 0.1404556092800184
0 - 3: 1.7919306344274455, 0.07427209352220686
1 - 2: -1.0528033246768713, 0.29337943063500393
1 - 3: 0.18106799639951288, 0.8564510623186123
2 - 3: 0.7243660400702261, 0.4694729237768852
